# Boxing Biomechanics — Análisis Unificado
**Pipeline automático: Detección de eventos → Cinemática → Variables de Fuerza → Resumen**

Solo necesitas cambiar `FILENAME` en la celda de configuración.

In [ ]:
# ─────────────────────────────────────────────
# CONFIGURACIÓN — Solo edita esta celda
# ─────────────────────────────────────────────
FILENAME = 'K001_J left hand pad.xlsx'  # Nombre del archivo Excel

NUM_EVENTS       = 5      # Número de eventos (golpes) a detectar
FORCE_THRESHOLD  = 300    # N — umbral mínimo de fuerza para considerar un pico
MIN_TIME_SEP     = 0.5    # s — separación mínima entre eventos
WINDOW_SIZE      = 0.4    # s — ventana alrededor del pico (±200 ms)
ACC_THRESHOLD    = 12     # m/s² — umbral para detectar inicio del golpe
ONSET_THRESHOLD  = 20     # N — umbral de fuerza para detectar inicio/fin del contacto

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy.signal import find_peaks
import warnings
warnings.filterwarnings('ignore')

# ── Constantes ──────────────────────────────────
MILLI_G_TO_MPS2 = 9.80665 / 1000  # milli-g → m/s²

# ── Carga y preprocesado ─────────────────────────
raw = pd.read_excel(FILENAME)

# Convertir aceleraciones de milli-g a m/s²
raw[['1x', '1y', '1z']] = raw[['1x', '1y', '1z']] * MILLI_G_TO_MPS2

# Magnitudes resultantes
raw['resultant_acceleration'] = np.sqrt(raw['1x']**2 + raw['1y']**2 + raw['1z']**2)
raw['resultant_force']        = np.sqrt(raw['fx']**2 + raw['fy']**2 + raw['fz']**2)

print(f"Archivo: {FILENAME}")
print(f"Filas: {len(raw):,}  |  Tiempo: {raw['Time'].min():.3f}s – {raw['Time'].max():.3f}s")
print(f"Fuerza máx: {raw['resultant_force'].max():.1f} N  |  Aceleración máx: {raw['resultant_acceleration'].max():.1f} m/s²")

## Detección de Eventos

In [ ]:
def find_top_peaks(data, num_peaks=NUM_EVENTS, min_time_sep=MIN_TIME_SEP, force_thresh=FORCE_THRESHOLD):
    """Detecta los N picos de mayor fuerza separados al menos min_time_sep segundos."""
    valid = data[data['resultant_force'] >= force_thresh]
    sorted_data = valid.sort_values('resultant_force', ascending=False)
    peaks = []
    for idx, row in sorted_data.iterrows():
        if len(peaks) >= num_peaks:
            break
        diffs = [abs(row['Time'] - data.loc[p, 'Time']) for p in peaks]
        if all(d >= min_time_sep for d in diffs):
            peaks.append(idx)
    # Ordenar por tiempo
    peaks.sort(key=lambda i: data.loc[i, 'Time'])
    return peaks

peak_indices = find_top_peaks(raw)
print(f"Eventos detectados: {len(peak_indices)}")
for i, idx in enumerate(peak_indices, 1):
    t = raw.loc[idx, 'Time']
    f = raw.loc[idx, 'resultant_force']
    print(f"  Evento {i}: t = {t:.3f} s  |  F = {f:.1f} N")

In [ ]:
# ── Visualización de todos los eventos ──────────
fig, axes = plt.subplots(2, 1, figsize=(16, 8), sharex=False)

# Fuerza resultante con ventanas
axes[0].plot(raw['Time'], raw['resultant_force'], color='steelblue', linewidth=0.8, label='Fuerza Resultante')
axes[0].set_ylabel('Fuerza (N)')
axes[0].set_title('Detección de Eventos — Fuerza Resultante')
axes[0].grid(True, alpha=0.3)

colors = plt.cm.tab10.colors
for i, idx in enumerate(peak_indices, 1):
    pt = raw.loc[idx, 'Time']
    pf = raw.loc[idx, 'resultant_force']
    c = colors[i-1]
    axes[0].axvspan(pt - WINDOW_SIZE/2, pt + WINDOW_SIZE/2, alpha=0.2, color=c)
    axes[0].plot(pt, pf, 'X', markersize=12, color=c, label=f'Evento {i} ({pt:.2f}s, {pf:.0f}N)')

axes[0].legend(fontsize=8, loc='upper right')

# Aceleración resultante
axes[1].plot(raw['Time'], raw['resultant_acceleration'], color='darkorange', linewidth=0.8, label='Aceleración Resultante')
axes[1].set_ylabel('Aceleración (m/s²)')
axes[1].set_xlabel('Tiempo (s)')
axes[1].set_title('Aceleración Resultante del Puño (Sensor 1)')
axes[1].grid(True, alpha=0.3)
axes[1].legend()

plt.tight_layout()
plt.savefig('events_overview.png', dpi=150, bbox_inches='tight')
plt.show()
print("Guardado: events_overview.png")

## Análisis por Evento — Cinemática + Fuerza

In [ ]:
def integrate_velocity(acc_values, times):
    """Integra aceleración para obtener velocidad (trapecio)."""
    vel = np.zeros(len(acc_values))
    for i in range(1, len(vel)):
        dt = times[i] - times[i-1]
        vel[i] = vel[i-1] + acc_values[i-1] * dt
    return vel

def find_strike_onset(force_series, peak_idx_local, threshold=ONSET_THRESHOLD):
    """Retrocede desde el pico buscando cuándo la fuerza cae bajo el umbral."""
    for i in range(peak_idx_local, -1, -1):
        if force_series.iloc[i] < threshold:
            return i
    return 0

def find_strike_end(force_series, peak_idx_local, threshold=ONSET_THRESHOLD):
    """Avanza desde el pico buscando cuándo la fuerza cae bajo el umbral."""
    for i in range(peak_idx_local, len(force_series)):
        if force_series.iloc[i] < threshold:
            return i
    return len(force_series) - 1

def find_acc_onset(acc_series, threshold=ACC_THRESHOLD):
    """Encuentra el primer índice donde la aceleración supera el umbral."""
    for i, val in enumerate(acc_series):
        if val > threshold:
            return i
    return 0

print("Funciones auxiliares definidas. ✓")

In [ ]:
all_results = []
event_dataframes = {}

for ev_num, peak_idx in enumerate(peak_indices, 1):
    peak_time = raw.loc[peak_idx, 'Time']
    
    # ── Extraer ventana del evento ───────────────
    t_start = peak_time - WINDOW_SIZE / 2
    t_end   = peak_time + WINDOW_SIZE / 2
    ev = raw[(raw['Time'] >= t_start) & (raw['Time'] <= t_end)].copy()
    ev.reset_index(drop=True, inplace=True)
    ev['Time_rel'] = ev['Time'] - t_start  # tiempo relativo desde inicio ventana
    
    # ── Velocidad por integración ────────────────
    times = ev['Time'].values
    ev['vel_x'] = integrate_velocity(ev['1x'].values, times)
    ev['vel_y'] = integrate_velocity(ev['1y'].values, times)
    ev['vel_z'] = integrate_velocity(ev['1z'].values, times)
    ev['resultant_velocity'] = np.sqrt(ev['vel_x']**2 + ev['vel_y']**2 + ev['vel_z']**2)
    
    # ── Índices clave ────────────────────────────
    local_peak_force_idx = ev['resultant_force'].idxmax()
    acc_onset_idx        = find_acc_onset(ev['resultant_acceleration'])
    force_onset_idx      = find_strike_onset(ev['resultant_force'], local_peak_force_idx)
    force_end_idx        = find_strike_end(ev['resultant_force'], local_peak_force_idx)
    
    # ── Métricas cinemáticas ─────────────────────
    t_acc_onset    = ev.loc[acc_onset_idx, 'Time']
    t_peak_force   = ev.loc[local_peak_force_idx, 'Time']
    strike_duration_kinematic = t_peak_force - t_acc_onset  # s
    velocity_at_peak_force    = ev.loc[local_peak_force_idx, 'resultant_velocity']
    max_velocity              = ev['resultant_velocity'].max()
    acc_at_peak_force         = ev.loc[local_peak_force_idx, 'resultant_acceleration']
    
    # ── Métricas de fuerza ───────────────────────
    peak_force    = ev.loc[local_peak_force_idx, 'resultant_force']
    force_onset_val = ev.loc[force_onset_idx, 'resultant_force']
    net_peak_force  = peak_force - force_onset_val
    
    t_force_onset = ev.loc[force_onset_idx, 'Time']
    t_force_end   = ev.loc[force_end_idx, 'Time']
    time_to_peak_ms  = (t_peak_force - t_force_onset) * 1000
    contact_dur_ms   = (t_force_end - t_force_onset) * 1000
    
    # Impulso
    contact_data = ev.loc[force_onset_idx:force_end_idx]
    impulse_total = np.trapz(contact_data['resultant_force'], x=contact_data['Time'])
    impulse_to_peak = np.trapz(
        ev.loc[force_onset_idx:local_peak_force_idx, 'resultant_force'],
        x=ev.loc[force_onset_idx:local_peak_force_idx, 'Time']
    )
    
    # RFD y Jerk
    dt_mean = ev['Time'].diff().mean()
    ev['RFD']  = ev['resultant_force'].diff() / dt_mean
    ev['Jerk'] = ev['RFD'].diff() / dt_mean
    max_rfd  = ev['RFD'].max()
    max_jerk = ev['Jerk'].max()
    
    # Fuerza a 10ms y 20ms desde inicio de contacto
    def force_at_ms(ms):
        target = t_force_onset + ms / 1000.0
        idx = (ev['Time'] - target).abs().idxmin()
        return ev.loc[idx, 'resultant_force']
    
    f_10ms = force_at_ms(10)
    f_20ms = force_at_ms(20)
    rfd_0_10 = (f_10ms - force_onset_val) / 0.010 if 0.010 > 0 else 0
    rfd_0_20 = (f_20ms - force_onset_val) / 0.020 if 0.020 > 0 else 0
    
    # ── Guardar resultados ───────────────────────
    result = {
        'Evento': ev_num,
        'Tiempo Pico (s)': round(peak_time, 3),
        'Fuerza Pico (N)': round(peak_force, 2),
        'Fuerza Neta Pico (N)': round(net_peak_force, 2),
        'Tiempo hasta Pico (ms)': round(time_to_peak_ms, 1),
        'Duración Contacto (ms)': round(contact_dur_ms, 1),
        'Impulso Total (N·s)': round(impulse_total, 3),
        'Impulso hasta Pico (N·s)': round(impulse_to_peak, 3),
        'RFD Máx (N/s)': round(max_rfd, 0),
        'Jerk Máx (N/s²)': round(max_jerk, 0),
        'Fuerza @ 10ms (N)': round(f_10ms, 2),
        'RFD 0-10ms (N/s)': round(rfd_0_10, 0),
        'Fuerza @ 20ms (N)': round(f_20ms, 2),
        'RFD 0-20ms (N/s)': round(rfd_0_20, 0),
        'Aceleración en Pico (m/s²)': round(acc_at_peak_force, 3),
        'Velocidad en Pico (m/s)': round(velocity_at_peak_force, 3),
        'Velocidad Máx (m/s)': round(max_velocity, 3),
        'Duración Golpe Cinemática (s)': round(strike_duration_kinematic, 3),
    }
    all_results.append(result)
    event_dataframes[ev_num] = ev
    print(f"✓ Evento {ev_num} procesado — Fuerza: {peak_force:.0f} N  |  Velocidad en impacto: {velocity_at_peak_force:.2f} m/s")

print("\nAnálisis completo.")

## Gráficos por Evento

In [ ]:
for ev_num, (result, peak_idx) in enumerate(zip(all_results, peak_indices), 1):
    ev = event_dataframes[ev_num]
    peak_time = raw.loc[peak_idx, 'Time']
    t_force_onset = raw.loc[peak_idx, 'Time'] - WINDOW_SIZE/2  # approximation for label
    
    # Recalcular índices locales para gráfico
    local_peak_f_idx = ev['resultant_force'].idxmax()
    force_onset_idx  = find_strike_onset(ev['resultant_force'], local_peak_f_idx)
    force_end_idx    = find_strike_end(ev['resultant_force'], local_peak_f_idx)
    acc_onset_idx    = find_acc_onset(ev['resultant_acceleration'])
    
    t_rel = ev['Time_rel'] * 1000  # ms
    
    fig = plt.figure(figsize=(16, 12))
    gs = gridspec.GridSpec(3, 2, figure=fig, hspace=0.45, wspace=0.3)
    
    ax_force   = fig.add_subplot(gs[0, 0])
    ax_acc     = fig.add_subplot(gs[0, 1])
    ax_vel     = fig.add_subplot(gs[1, 0])
    ax_vel_xyz = fig.add_subplot(gs[1, 1])
    ax_rfd     = fig.add_subplot(gs[2, 0])
    ax_table   = fig.add_subplot(gs[2, 1])
    
    fig.suptitle(f'Evento {ev_num} — t = {peak_time:.3f}s | Fuerza Pico: {result["Fuerza Pico (N)"]:.0f} N',
                 fontsize=14, fontweight='bold')
    
    # ── Fuerza ──────────────────────────────────
    ax_force.plot(t_rel, ev['resultant_force'], color='steelblue', linewidth=2)
    ax_force.fill_between(t_rel, ev['resultant_force'], alpha=0.15, color='steelblue')
    ax_force.axvline(t_rel.iloc[force_onset_idx], color='green', ls='--', label='Inicio contacto')
    ax_force.axvline(t_rel.iloc[local_peak_f_idx], color='red', ls='--', label='Pico fuerza')
    ax_force.axvline(t_rel.iloc[force_end_idx], color='black', ls='--', label='Fin contacto')
    ax_force.set_title('Fuerza Resultante')
    ax_force.set_xlabel('Tiempo (ms)')
    ax_force.set_ylabel('Fuerza (N)')
    ax_force.legend(fontsize=7)
    ax_force.grid(True, alpha=0.3)
    
    # ── Aceleración ─────────────────────────────
    ax_acc.plot(t_rel, ev['resultant_acceleration'], color='darkorange', linewidth=2)
    ax_acc.axvline(t_rel.iloc[acc_onset_idx], color='purple', ls='--', label='Inicio golpe (acc)')
    ax_acc.axvline(t_rel.iloc[local_peak_f_idx], color='red', ls='--', label='Pico fuerza')
    ax_acc.set_title('Aceleración Resultante')
    ax_acc.set_xlabel('Tiempo (ms)')
    ax_acc.set_ylabel('Aceleración (m/s²)')
    ax_acc.legend(fontsize=7)
    ax_acc.grid(True, alpha=0.3)
    
    # ── Velocidad resultante ─────────────────────
    ax_vel.plot(t_rel, ev['resultant_velocity'], color='purple', linewidth=2, label='Vel. Resultante')
    ax_vel.axvline(t_rel.iloc[local_peak_f_idx], color='red', ls='--', label='Pico fuerza')
    vel_at_peak = ev.loc[local_peak_f_idx, 'resultant_velocity']
    ax_vel.scatter(t_rel.iloc[local_peak_f_idx], vel_at_peak, color='red', s=100, zorder=5)
    ax_vel.annotate(f'{vel_at_peak:.2f} m/s', (t_rel.iloc[local_peak_f_idx], vel_at_peak),
                    xytext=(8, -10), textcoords='offset points', fontsize=9, color='red')
    ax_vel.set_title('Velocidad Resultante del Puño')
    ax_vel.set_xlabel('Tiempo (ms)')
    ax_vel.set_ylabel('Velocidad (m/s)')
    ax_vel.legend(fontsize=7)
    ax_vel.grid(True, alpha=0.3)
    
    # ── Velocidad por ejes ───────────────────────
    ax_vel_xyz.plot(t_rel, ev['vel_x'], label='Vel X', color='red', linewidth=1.5)
    ax_vel_xyz.plot(t_rel, ev['vel_y'], label='Vel Y', color='green', linewidth=1.5)
    ax_vel_xyz.plot(t_rel, ev['vel_z'], label='Vel Z', color='blue', linewidth=1.5)
    ax_vel_xyz.axvline(t_rel.iloc[local_peak_f_idx], color='red', ls='--', alpha=0.5)
    ax_vel_xyz.set_title('Velocidad por Ejes')
    ax_vel_xyz.set_xlabel('Tiempo (ms)')
    ax_vel_xyz.set_ylabel('Velocidad (m/s)')
    ax_vel_xyz.legend(fontsize=7)
    ax_vel_xyz.grid(True, alpha=0.3)
    
    # ── RFD ─────────────────────────────────────
    rfd_clean = ev['RFD'].dropna()
    ax_rfd.plot(t_rel.iloc[rfd_clean.index], rfd_clean, color='crimson', linewidth=1.5)
    ax_rfd.axvline(t_rel.iloc[local_peak_f_idx], color='red', ls='--', alpha=0.7)
    ax_rfd.set_title('Rate of Force Development (RFD)')
    ax_rfd.set_xlabel('Tiempo (ms)')
    ax_rfd.set_ylabel('RFD (N/s)')
    ax_rfd.grid(True, alpha=0.3)
    
    # ── Tabla de métricas ────────────────────────
    ax_table.axis('off')
    table_data = [
        ['Fuerza Pico', f"{result['Fuerza Pico (N)']:.1f}", 'N'],
        ['Fuerza Neta Pico', f"{result['Fuerza Neta Pico (N)']:.1f}", 'N'],
        ['Tiempo hasta Pico', f"{result['Tiempo hasta Pico (ms)']:.1f}", 'ms'],
        ['Duración Contacto', f"{result['Duración Contacto (ms)']:.1f}", 'ms'],
        ['Impulso Total', f"{result['Impulso Total (N·s)']:.3f}", 'N·s'],
        ['Impulso hasta Pico', f"{result['Impulso hasta Pico (N·s)']:.3f}", 'N·s'],
        ['RFD Máx', f"{result['RFD Máx (N/s)']:.0f}", 'N/s'],
        ['F @ 10ms', f"{result['Fuerza @ 10ms (N)']:.1f}", 'N'],
        ['RFD 0-10ms', f"{result['RFD 0-10ms (N/s)']:.0f}", 'N/s'],
        ['F @ 20ms', f"{result['Fuerza @ 20ms (N)']:.1f}", 'N'],
        ['Vel. en Impacto', f"{result['Velocidad en Pico (m/s)']:.3f}", 'm/s'],
        ['Vel. Máxima', f"{result['Velocidad Máx (m/s)']:.3f}", 'm/s'],
        ['Aceleración en Pico', f"{result['Aceleración en Pico (m/s²)']:.1f}", 'm/s²'],
        ['Dur. Golpe (cinematic)', f"{result['Duración Golpe Cinemática (s)']*1000:.1f}", 'ms'],
    ]
    tbl = ax_table.table(cellText=table_data,
                         colLabels=['Métrica', 'Valor', 'Unidad'],
                         loc='center', cellLoc='left',
                         colColours=['#e8e8e8']*3)
    tbl.auto_set_font_size(False)
    tbl.set_fontsize(8)
    tbl.scale(1, 1.35)
    ax_table.set_title(f'Métricas Evento {ev_num}', pad=10)
    
    plt.savefig(f'event_{ev_num}_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    print(f"Guardado: event_{ev_num}_analysis.png")

## Tabla Resumen — Todos los Eventos

In [ ]:
summary_df = pd.DataFrame(all_results)
summary_df = summary_df.set_index('Evento')

# ── Mostrar tabla formateada ─────────────────────
with pd.option_context('display.max_columns', None, 'display.width', 200, 'display.float_format', '{:.2f}'.format):
    display(summary_df.T)

print("\n── Estadísticas Descriptivas ──")
with pd.option_context('display.float_format', '{:.2f}'.format):
    display(summary_df[[
        'Fuerza Pico (N)', 'Fuerza Neta Pico (N)', 'Tiempo hasta Pico (ms)',
        'Duración Contacto (ms)', 'Impulso Total (N·s)', 'RFD Máx (N/s)',
        'Velocidad en Pico (m/s)', 'Velocidad Máx (m/s)'
    ]].agg(['mean', 'std', 'min', 'max']))

In [ ]:
# ── Gráfico comparativo entre eventos ───────────
metrics_to_plot = [
    ('Fuerza Pico (N)', 'Fuerza Pico (N)', 'steelblue'),
    ('Velocidad en Pico (m/s)', 'Velocidad en Impacto (m/s)', 'darkorange'),
    ('RFD Máx (N/s)', 'RFD Máx (N/s)', 'crimson'),
    ('Impulso Total (N·s)', 'Impulso Total (N·s)', 'green'),
    ('Tiempo hasta Pico (ms)', 'Tiempo hasta Pico (ms)', 'purple'),
    ('Duración Contacto (ms)', 'Duración Contacto (ms)', 'brown'),
]

fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()
eventos = summary_df.index.tolist()

for ax, (col, label, color) in zip(axes, metrics_to_plot):
    vals = summary_df[col]
    bars = ax.bar([f'E{i}' for i in eventos], vals, color=color, alpha=0.8, edgecolor='white')
    ax.set_title(label, fontsize=10, fontweight='bold')
    ax.set_ylabel(label)
    ax.grid(axis='y', alpha=0.3)
    # Valor encima de cada barra
    for bar, val in zip(bars, vals):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() * 1.01,
                f'{val:.1f}', ha='center', va='bottom', fontsize=8)

plt.suptitle('Comparación entre Eventos', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('events_comparison.png', dpi=150, bbox_inches='tight')
plt.show()
print("Guardado: events_comparison.png")

## Exportar Resultados

In [ ]:
base_name = FILENAME.rsplit('.', 1)[0]
output_xlsx = f'{base_name}_resultados.xlsx'

with pd.ExcelWriter(output_xlsx, engine='openpyxl') as writer:
    # Hoja resumen
    summary_df.to_excel(writer, sheet_name='Resumen')
    
    # Hoja de estadísticas
    stats_df = summary_df[[
        'Fuerza Pico (N)', 'Fuerza Neta Pico (N)', 'Tiempo hasta Pico (ms)',
        'Duración Contacto (ms)', 'Impulso Total (N·s)', 'RFD Máx (N/s)',
        'Velocidad en Pico (m/s)', 'Velocidad Máx (m/s)'
    ]].agg(['mean', 'std', 'min', 'max'])
    stats_df.to_excel(writer, sheet_name='Estadísticas')
    
    # Una hoja por evento
    for ev_num, ev_df in event_dataframes.items():
        ev_df.to_excel(writer, sheet_name=f'Evento_{ev_num}', index=False)

print(f"Exportado: {output_xlsx}")
print(f"Hojas: Resumen, Estadísticas, Evento_1 … Evento_{len(event_dataframes)}")